# Initialisation

In [ ]:
%matplotlib ipympl
from matplotlib import pyplot as plt
import numpy as np
#import scipy
from scipy.special import zeta
from time import time
from astropy import cosmology as cosmo
from astropy import units as u
from astropy import constants as c
from astropy.visualization import quantity_support
quantity_support()

# Models

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from models import flcdm, coasting

# Plot cosmic evolution

In [ ]:
fig_name = 'evolution'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(8, 8))
axes = fig.subplots(nrows=2, ncols=1, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )
alpha = 1

ax = axes[0, 0]
ax.set_ylabel(r'density $\rho$ [M$_\odot$ Mpc$^{-3}$]')

def mass_to_E(m):
    return((m*c.c**2 * u.Msun/u.Mpc**3).to_value(u.GeV/u.cm**3))
def E_to_mass(E):
    return((E/c.c**2 * u.GeV/u.cm**3).to_value(u.Msun/u.Mpc**3))
secax = ax.secondary_yaxis('right', functions=(mass_to_E, E_to_mass))
secax.set_ylabel(r'energy density $\rho c^2$ [GeV cm$^{-3}$]')


ax.set_yscale('log')
ax.set_xscale('log')

legends = []
for model, title, ls, bbox in zip([flcdm, coasting],
                                  [r'Flat $\Lambda$CDM', 'Coasting'],
                                  ['--', '-'],
                                  [(0.45, 0.5, 0.5, 0.5), (0.72, 0.5, 0.5, 0.5)],
                                 ):
    t = model.t.to_value(u.yr)
    rho_c = model.rho_crit_t.to_value(u.Msun/u.Mpc**3)
    rho_de = np.exp(model.ln_Ode_t) * rho_c
    rho_m = np.exp(model.ln_Om_t) * rho_c
    rho_b = rho_m * model.Ob0/model.Om0
    rho_gamma = np.exp(model.ln_Ogamma_t) *rho_c
    
    l1, = ax.plot(t, rho_de, c='r', ls=ls, alpha=alpha, label='dark energy')
    l2, = ax.plot(t, rho_m - rho_b, c='k', ls=ls, alpha=alpha, label='dark matter')
    l3, = ax.plot(t, rho_b, c='c', ls=ls, alpha=alpha, label='baryons (atoms)')
    l4, = ax.plot(t, rho_gamma, c='y', ls=ls, alpha=alpha, label='photons')
    legends.append(ax.legend(handles=[l1, l2, l3, l4], title=title, loc='upper left', bbox_to_anchor=bbox))

ax.add_artist(legends[0])
ax.set_xlim(3e3, 3e10)
ax.set_ylim(3e6, 3e25)
ax.set_yticks(np.logspace(7, 25, 19), minor=True, labels='')

ax = axes[1, 0]
ax.set_ylabel('ionisation fraction $x$')

ax.plot(flcdm.t.to_value(u.yr), flcdm.x_ion_t, 'c--', alpha=alpha)
ax.plot(coasting.t.to_value(u.yr), coasting.x_ion_t, 'c-', alpha=alpha)

ax.plot(flcdm.t_cmb.to_value(u.yr), np.interp(flcdm.t_cmb, flcdm.t, flcdm.x_ion_t), 'cd', fillstyle='none',
           label=r'Flat $\Lambda$CDM: $t_{\rm CMB} \sim$'+f'{flcdm.t_cmb.to_value(u.Myr):.3f} Myr')
ax.plot(coasting.t_cmb.to_value(u.yr), np.interp(coasting.t_cmb, coasting.t, coasting.x_ion_t), 'cd',
        label=r'Coasting: $t_{\rm CMB} \sim$'+f'{coasting.t_cmb.to_value(u.Myr):.3f} Myr')

l = ax.legend().get_lines()
l[0].set_ls('--')
l[1].set_ls('-')
#h.set_visible(False)

ax.set_xscale('log')
ax.set_xlabel('cosmic time $t$ [yr]')


for ax in axes.flatten():
    #ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
    ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
    ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
    ax.grid(True, which='both')
    #ax.axvline(t_cmb_p18, c='r', ls='--')
    #ax.axvline(t_cmb_model, c='k', ls='-')
fig.set_tight_layout(True)
plt.show()
plt.savefig('figures/evolution.pdf')